# Zeran's mapping exercise
Choose one of the map ideas below, processed the data with GeoPandas and export the data to QGIS for presentation. *Hint: You might need to change data types using the `dtype` argument when importing files so that you can merge with FIPS and ZIP codes effectively.*

#### Load python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [3]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

### Election map
#### What percentage of vote did Biden/Trump receive in California counties?

In [4]:
# Values: https://raw.githubusercontent.com/stiles/notebooks/master/elections/data/ca_counties_election_results_2020.csv
# Geo: https://raw.githubusercontent.com/stiles/usc/main/data/geo/counties_simple.geojson

#### Read election results

In [5]:
election_df = pd.read_csv(
    "https://raw.githubusercontent.com/stiles/notebooks/master/elections/data/ca_counties_election_results_2020.csv"
)

In [6]:
election_df.head()

,state_name,county_fips,county_name,votes_gop,votes_dem,total_votes,diff,per_gop,per_dem,per_point_diff,winner
0,California,1,Alameda,136309,617659,769864,-481350,0.177056,0.802296,0.625240,Biden
1,California,3,Alpine,244,476,741,-232,0.329285,0.642375,0.313090,Biden
2,California,5,Amador,13585,8153,22302,5432,0.609138,0.365573,0.243566,Trump
3,California,7,Butte,48730,50426,102042,-1696,0.477548,0.494169,0.016621,Biden
4,California,9,Calaveras,16518,10046,27164,6472,0.608084,0.369828,0.238257,Trump


In [7]:
election_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   state_name      58 non-null     object 
 1   county_fips     58 non-null     int64  
 2   county_name     58 non-null     object 
 3   votes_gop       58 non-null     int64  
 4   votes_dem       58 non-null     int64  
 5   total_votes     58 non-null     int64  
 6   diff            58 non-null     int64  
 7   per_gop         58 non-null     float64
 8   per_dem         58 non-null     float64
 9   per_point_diff  58 non-null     float64
 10  winner          58 non-null     object 
dtypes: float64(3), int64(5), object(3)
memory usage: 5.1+ KB


In [8]:
election_df = pd.read_csv(
    "https://raw.githubusercontent.com/stiles/notebooks/master/elections/data/ca_counties_election_results_2020.csv",
    dtype={"county_fips": str},
)

#### Read county boundaries

In [9]:
ca_counties_gdf = gpd.read_file(
    "https://raw.githubusercontent.com/stiles/usc/main/data/geo/counties_simple.geojson",
    dtype={"county_fips": str},
)

In [10]:
ca_counties_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   county_name  58 non-null     object  
 1   county_fips  58 non-null     object  
 2   geometry     58 non-null     geometry
dtypes: geometry(1), object(2)
memory usage: 1.5+ KB


#### Merge the two 

In [11]:
ca_presidential = pd.merge(
    ca_counties_gdf, election_df, on=["county_fips", "county_name"]
)

In [12]:
ca_presidential.head()

,county_name,county_fips,geometry,state_name,votes_gop,votes_dem,total_votes,diff,per_gop,per_dem,per_point_diff,winner
0,Alameda,001,"POLYGON ((-122.27125 37.90503, -122.27024 37.9...",California,136309,617659,769864,-481350,0.177056,0.802296,0.625240,Biden
1,Alpine,003,"POLYGON ((-119.58667 38.71420, -119.58653 38.7...",California,244,476,741,-232,0.329285,0.642375,0.313090,Biden
2,Amador,005,"POLYGON ((-120.07246 38.70276, -120.07249 38.6...",California,13585,8153,22302,5432,0.609138,0.365573,0.243566,Trump
3,Butte,007,"POLYGON ((-121.07661 39.59729, -121.07945 39.5...",California,48730,50426,102042,-1696,0.477548,0.494169,0.016621,Biden
4,Calaveras,009,"POLYGON ((-120.01792 38.43586, -120.01788 38.4...",California,16518,10046,27164,6472,0.608084,0.369828,0.238257,Trump


In [13]:
ca_presidential.to_file("ca_presidential.geojson", driver="GeoJSON")

---

### Vehicle map
##### What's the Tesla ownership rate by LA ZIP Codes?

In [14]:
# Values: https://raw.githubusercontent.com/stiles/notebooks/master/california-fuel-vehicles/output/tesla_zips_slim.csv
# Geo: https://raw.githubusercontent.com/stiles/notebooks/master/zips/data/processed/ca_zips_simple.geojson.json

#### Read Tesla data

In [15]:
tesla_df = pd.read_csv(
    "https://raw.githubusercontent.com/stiles/notebooks/master/california-fuel-vehicles/output/tesla_zips_slim.csv"
)

In [17]:
tesla_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 882 entries, 0 to 881
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   zip               882 non-null    int64  
 1   name              882 non-null    object 
 2   teslas            882 non-null    int64  
 3   all_vehicles      882 non-null    int64  
 4   tesla_rate_1k     882 non-null    float64
 5   avg_house_income  882 non-null    int64  
 6   avg_net_worth     882 non-null    int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 48.4+ KB


In [18]:
tesla_df = pd.read_csv(
    "https://raw.githubusercontent.com/stiles/notebooks/master/california-fuel-vehicles/output/tesla_zips_slim.csv",
    dtype={"zip": str},
)

#### Read ZIP Codes geo file

In [19]:
zip_gdf = gpd.read_file(
    "https://raw.githubusercontent.com/stiles/notebooks/master/zips/data/processed/ca_zips_simple.geojson.json"
)

In [20]:
zip_gdf.head()

,id,name,county_name,geometry
0,90001,Los Angeles,Los Angeles County,"POLYGON ((-118.24329 33.98901, -118.25996 33.9..."
1,90002,Los Angeles,Los Angeles County,"POLYGON ((-118.23085 33.95772, -118.23160 33.9..."
2,90003,Los Angeles,Los Angeles County,"POLYGON ((-118.28268 33.98926, -118.28383 33.9..."
3,90004,Los Angeles,Los Angeles County,"MULTIPOLYGON (((-118.33852 34.06891, -118.3371..."
4,90005,Los Angeles,Los Angeles County,"POLYGON ((-118.31879 34.05514, -118.30658 34.0..."


#### Merge the two

#### Export merged geo file

---